# Mejorando el accuracy del arbol
## Oversampling y Undersampling
A pesar de usar estas tecnicas la precision del modelo parece no cambiar, esto es debido a que la informacion no esta balanceada. Ya que aprox mas del 80% de los pacientes no han desarrollado alguna enfermedad.
Esto se soluciona aplicando sobre muestreo y sub-muestreo. En este caso se haran ambos.

In [3]:
#Librerias 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Arbol de decision
from sklearn.tree import DecisionTreeClassifier#-
from sklearn.model_selection import train_test_split

#Para el bosque aleatorio.
from sklearn.ensemble import RandomForestClassifier

#Para el reporte
from sklearn.metrics import classification_report, accuracy_score,make_scorer#-
from sklearn.model_selection import ParameterGrid, GridSearchCV


In [2]:
#Importacion del dataset
dataset = pd.read_csv("framinghamClean.csv")
dataset.drop(["Unnamed: 0"], axis=1, inplace=True)
dataset.head()
#dataset.shape

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


## Preparando la informacion

In [ ]:
X = 